# **PROCESO DE LIMPIEZA DE *SCRAPPING***

In [113]:
import pandas as pd # Manipulación df
import os # Directorios
import re # Manipulación de cadenas de texto (expresiones regulares)
import unicodedata# Función para eliminar acentos
from datetime import datetime, date
from currency_converter import CurrencyConverter # convertidor de monedas

# Cargar archivo

In [150]:
directorio_actual = os.getcwd() # Directorio actual de trabajo
# Ruta relativa al archivo CSV
csv_filename = 'lamudi-Mayo-QueretaroInmuebles .xlsx'
csv_dir = os.path.join(directorio_actual)#, 'datos'
csv_path = os.path.join(csv_dir, csv_filename)

Leer archivo sin nombre en columnas:

In [151]:
#column_names=['precio','denominacion','propiedad','metros_total','metros_construido','tiempo_de_publicacion','tipo','estacionamientos','recamaras','banos','medio_banos','seguridad_privada','ubicacion','url','descripcion']
df = pd.read_excel(csv_path)#, names=column_names
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9630 entries, 0 to 9629
Data columns (total 15 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   precio                 9630 non-null   int64 
 1   denominacion           9630 non-null   object
 2   propiedad              9630 non-null   object
 3   metros_total           9630 non-null   int64 
 4   metros_construido      9630 non-null   int64 
 5   tiempo_de_publicacion  9630 non-null   object
 6   tipo                   9629 non-null   object
 7   estacionamientos       9630 non-null   int64 
 8   recamaras              9630 non-null   int64 
 9   banos                  9630 non-null   int64 
 10  medio_banos            9630 non-null   int64 
 11  seguridad_privada      9630 non-null   object
 12  ubicacion              9630 non-null   object
 13  url                    9630 non-null   object
 14  descripcion            9630 non-null   object
dtypes: int64(7), object(8

In [137]:
df.head()

,precio,denominacion,propiedad,metros_total,metros_construido,tiempo_de_publicacion,tipo,estacionamientos,recamaras,banos,medio_banos,seguridad_privada,ubicacion,url,descripcion
0,400000,MXN,"VENTA DE DEPARTAMENTO, SANTIAGO DE QUERÉTARO.",67,67,2024-02-01 00:00:00,Departamento,2,2,1,0,No,"El Rocío, Querétaro, Querétaro",https://www.lamudi.com.mx//detalle/41032-73-8df481948b19-342b-23dceb56-9c14-4cc6,"APROVECHA LA OPORTUNIDAD DE ADQUIRIR TU PROPIA CASA. EN UNA EXCELENTE UBICACION, CERCA DE CENTROS COMERCIALES, MERCADOS, FARMACIAS, ESCUELAS Y HOSPITALES, ADEMAS DE FACIL ACCESO A AVENIDAS PRINCIPALES. SI TE INTERESA CONTACTAME. ✍🏻🏡"
1,440450,MXN,LA LOMA SANTIAGO DE QUERETARO QUERETARO,50,50,2023-07-28 00:00:00,Departamento,1,2,1,0,No,"Fracc Las Quintas 3ra Sección, Querétaro, Querétaro",https://www.lamudi.com.mx//detalle/41032-73-780376c9b4eb-c4db-ec486239-b72c-3318,"Lindo departamento ubicado dentro de una unidad en queretaro.Este departamento consta de 50m2 de construcción, este consta de sala, comedor, cocina, dos recamaras y un baño.Se encuentra muy cerca de P. Queretaro, tiene cercania con supermercados, tiendas de conveniencia.VALOR COMERCIAL:$641,920.96VALOR OPORTUNIDAD:$440,450.00- ÚNICAMENTE PAGO DE CONTADOExcelente oportunidad de inversión. Revisión previa de expediente.El pago se hace directamente a la institución financiera.Los precios están sujetos a cambio, sin previo aviso. Todo el proceso ante el notario público de su elección.Te asesoramos hasta la entrega física y legal del inmueble.Agenda TU cita en nuestra oficina.¡LA CALIDAD DE SU FUTURO ESTÁ BASADA EN INVERTIR SABIAMENTE EN EL PRESENTE!Manejamos Inmuebles ubicados en diferentes partes de la República Mexicana, listos para tu adquisición. EasyBroker ID: EB-NZ7782"
2,469000,MXN,"DEPARTAMENTO EN VENTA TOSCANA, QUERÉTARO",180,180,2023-09-12 00:00:00,Departamento,1,3,1,0,No,"Fraccionamiento Los Viñedos, Querétaro, Querétaro",https://www.lamudi.com.mx//detalle/41032-73-baf00df1e2aa-6a49-414a933-ac9d-4de1,"¿Enfadado de que te digan siempre lo mismo? Entrega en 8 o más meses, pago en dos exhibiciones, no puedo mostrarte nada, primero depósito, etc., ***NO PIERDAS MÁS EL TIEMPO***TE MUESTRO LA LEGALIDAD DE LA EMPRESA Y DE LA PROPIEDADAgenda una cita presencial o una llamada sin costo y habita tu casa rápidamente.Te ayudo con alternativas de pago, sólo si cuentas con CAPITAL PROPIO DE INICIO.**TU SACRIFICIO DE HOY ES TU ALEGRÍA Y TRANQUILIDAD DEL MAÑANA** G.C."
3,500000,MXN,Departamento en Venta,42,42,18 ene. 2024,Departamento,1,2,1,0,No,"Fraccionamiento Hacienda Santa Rosa, Querétaro, Querétaro",https://www.lamudi.com.mx//detalle/41032-73-93cd05a2c544-ee5b-ce045421-bdcd-3324,"Disfruta un departamento muy acogedor, excelente ubicación. Cuenta con excelentes opciones a transporte público. Consta de 2 recámaras, 1 baño y un lugar de estacionamiento."
4,575000,MXN,Departamento En Venta Ubicado En 220 Av. Belén Santiago de Querétaro,70,70,2023-11-09 00:00:00,Departamento,1,2,1,0,No,"Condominio Flores, Querétaro, Querétaro",https://www.lamudi.com.mx//detalle/41032-73-664c2fd3a69f-ba15-20374fd8-bb03-4ec9,"Bienvenido, Gran Oportunidad De Inversión En Departamento Con 2 Habitaciones, 1 Baño, 1 Estacionamiento y Fácil Acceso a Av. Importantes, Bancos, Escuelas, Hospitales, Plazas Comerciales y Más, Todo a Unos Minutos, Siempre a Tu Servicio, Br. Comenzamos???"


Leer archivo con nombre de columnas:

### Limpieza de Nulos

In [152]:
df.fillna({'precio': 0}, inplace=True) 
df.fillna({'propiedad': 'Propiedad sin título'}, inplace=True) 
df.fillna({'metros_total': 0}, inplace=True) 
df.fillna({'metros_construido': 0}, inplace=True) 
df.fillna({'tiempo_de_publicacion': '1 ene 1000'}, inplace=True) 
df.fillna({'tipo': 'Propiedad sin tipo de casa'}, inplace=True) 
df.fillna({'estacionamientos': 0}, inplace=True) 
df.fillna({'recamaras': 0}, inplace=True) 
df.fillna({'banos': 0}, inplace=True) 
df.fillna({'medio_banos': 0}, inplace=True) 
df.fillna({'seguridad_privada': 'No'}, inplace=True) 
df.fillna({'ubicacion': 'Propiedad sin ubicación'}, inplace=True) 
df.fillna({'url': '-'}, inplace=True) 
df.fillna({'descripcion': "Propiedad sin descripcion"}, inplace=True) 

## Estandarizar variables

In [117]:
df.columns

Index(['precio', 'denominacion', 'propiedad', 'metros_total',
       'metros_construido', 'tiempo_de_publicacion', 'tipo',
       'estacionamientos', 'recamaras', 'banos', 'medio_banos',
       'seguridad_privada', 'ubicacion', 'url', 'descripcion'],
      dtype='object')

#### propiedad

In [118]:
# Convierte todos los valores a strings, maneja NaN o None como cadenas vacías
df['propiedad'] = df['propiedad'].astype(str)
# Convertir a minúsculas
df['propiedad'] = df['propiedad'].str.lower()

import unicodedata# Función para eliminar acentos
def eliminar_acentos(texto):
    # Normaliza y elimina los acentos
    texto_normalizado = unicodedata.normalize('NFKD', texto)
    return ''.join(char for char in texto_normalizado if unicodedata.category(char) != 'Mn')
df['propiedad'] = df['propiedad'].apply(eliminar_acentos)

# Reemplazar Ñ
df['propiedad'] = df['propiedad'].str.replace('ñ', 'n')
df['propiedad'] = df['propiedad'].str.replace(r'[^a-zA-Z0-9\s]', '', regex=True)

#### UBICACIÓN

In [79]:
# Convierte todos los valores a strings, maneja NaN o None como cadenas vacías
df['ubicacion'] = df['ubicacion'].astype(str)
# Convertir a minúsculas
df['ubicacion'] = df['ubicacion'].str.lower()

import unicodedata# Función para eliminar acentos
def eliminar_acentos(texto):
    # Normaliza y elimina los acentos
    texto_normalizado = unicodedata.normalize('NFKD', texto)
    return ''.join(char for char in texto_normalizado if unicodedata.category(char) != 'Mn')
df['ubicacion'] = df['ubicacion'].apply(eliminar_acentos)

# Reemplazar Ñ
df['ubicacion'] = df['ubicacion'].str.replace('ñ', 'n')
df['ubicacion'] = df['ubicacion'].str.replace(r'[^a-zA-Z0-9\s]', '', regex=True)

#### Descripción

In [119]:
# Convierte todos los valores a strings, maneja NaN o None como cadenas vacías
df['descripcion'] = df['descripcion'].astype(str)
# Convertir a minúsculas
df['descripcion'] = df['descripcion'].str.lower()

import unicodedata# Función para eliminar acentos
def eliminar_acentos(texto):
    # Normaliza y elimina los acentos
    texto_normalizado = unicodedata.normalize('NFKD', texto)
    return ''.join(char for char in texto_normalizado if unicodedata.category(char) != 'Mn')
df['descripcion'] = df['descripcion'].apply(eliminar_acentos)

# Reemplazar Ñ
df['descripcion'] = df['descripcion'].str.replace('ñ', 'n')
df['descripcion'] = df['descripcion'].str.replace(r'[^a-zA-Z0-9\s]', '', regex=True)

____
# 1. Manipulación de precios

## a) Precio en descripción

In [153]:
def extraer_precio(descripcion):
    descripcion = descripcion.lower()
    # Buscar patrones
    precios = re.findall(r'\b(\d+[\.,]?\d*[\.,]?\d*)\s*($|mil pesos|mxn|precio|pesos|monto|Pesos Mexicanos|USD)', descripcion)
    precio = None
    for match in precios:
        numero, moneda = match
        numero = numero.replace(',', '').replace("'", '').replace(".",'')
        try:
            precio = float(numero)
        except ValueError:
            continue
        if precio:
            break
    return precio

# Denominación de la descripción
def extraer_denominacion(descripcion):
    descripcion = descripcion.lower()
    # Buscar patrones 
    denominaciones = re.findall(r'\b(mil|mxn|precio|pesos|monto|Mexicanos|Estadounidenses)\b', descripcion)
    return denominaciones[0] if denominaciones else None

df['precio_desc'] = df['descripcion'].apply(extraer_precio)
#df['denominacion_desc'] = df['descripcion'].apply(extraer_denominacion)

### Comparaciones

#### Precio [0 / 0]

In [71]:
#posibles = df[( (df['precio'] == 0 ) & (df['precio_desc'].isna()) )] 
# Configurar pandas para que no use notación científica en la impresión
#pd.set_option('display.float_format', '{:.2f}'.format)
# Imprimir las columnas especificadas del DataFrame filtrado
#print(posibles[['precio', 'denominacion', 'precio_desc', 'url' ]]) #'denominacion_desc'
# Restaurar la configuración original después de la impresión si es necesario
#pd.reset_option('display.float_format')

##### Proceso de reemplazo

In [154]:
# Filtrar los registros según las condiciones dadas
totales = df[( (df['precio'] == 0 ) & (df['precio_desc'].isna()) )] 
pd.set_option('display.max_colwidth', None) # Mostrar URLs completas
# Iterar sobre los registros filtrados
for idx, row in totales.iterrows():
    print(f"Registro {idx}:")
    print(row[['precio', 'precio_desc', 'url']])
    print("\n¿Qué acción te gustaría realizar?")
    print("1. Sustituir el valor manualmente")
    print("2. Pasar al siguiente registro")

    # Solicitar la opción al usuario
    option = input("Elige una opción (1 o 2): ")

    if option == '1':
        # Sustituir el valor manualmente
        new_value = input("Introduce el nuevo valor para 'precio': ")
        df.at[idx, 'precio'] = float(new_value)
    elif option == '2':
        # Pasar al siguiente registro
        continue
    else:
        print("Opción no válida. Pasando al siguiente registro...")
    
    print("\n---\n")

print("Proceso completado.")

Proceso completado.


Eliminar registros que no recuperaron su precio verdadero desde descripción

In [141]:
df = df[~( (df['precio'] == 0 ) & (df['precio_desc'].isna()) )] 

#### Precio [0 / #]

In [44]:
#posibles = df[( (df['precio'] == 0 ) & (df['precio_desc'].notna()) )] 
# Configurar pandas para que no use notación científica en la impresión
#pd.set_option('display.float_format', '{:.2f}'.format)
# Imprimir las columnas especificadas del DataFrame filtrado
#print(posibles[['precio', 'denominacion', 'precio_desc', 'url' ]]) #'denominacion_desc'
# Restaurar la configuración original después de la impresión si es necesario
#pd.reset_option('display.float_format')

##### Proceso de reemplazo

In [45]:
# Filtrar los registros según las condiciones dadas
totales = df[( (df['precio'] == 0 ) & (df['precio_desc'].notna()) )] 

# Iterar sobre los registros filtrados
for idx, row in totales.iterrows():
    print(f"Registro {idx}:")
    print(row[['precio', 'precio_desc', 'url']])
    print("\n¿Qué acción te gustaría realizar?")
    print("1. Sustituir el valor manualmente")
    print("2. Sustituir el valor de 'precio' por el de 'precio_desc'")
    print("3. Pasar al siguiente registro")

    # Solicitar la opción al usuario
    option = input("Elige una opción (1, 2, 3): ")

    if option == '1':
        # Sustituir el valor manualmente
        new_value = input("Introduce el nuevo valor para 'precio': ")
        df.at[idx, 'precio'] = float(new_value)
    elif option == '2':
        # Sustituir el valor de 'metros_total' por 'totales_desc'
        df.at[idx, 'precio'] = df.at[idx, 'precio_desc']
    elif option == '3':
        # Pasar al siguiente registro
        continue
    else:
        print("Opción no válida. Pasando al siguiente registro...")
    
    print("\n---\n")

print("Proceso completado.")


Proceso completado.


#### Precio [# / #]

In [46]:
#posibles_cont = df[( (df['precio'].notna()) & (df['precio_desc'].notna()) )] 
# Filtro dentro del estándar
#sustituir = posibles_cont[(posibles_cont['precio_desc'] >= 250000) & (posibles_cont['precio_desc'] <= 500000000)]

# Configurar pandas para que no use notación científica en la impresión
#pd.set_option('display.float_format', '{:.2f}'.format)
# Imprimir las columnas especificadas del DataFrame filtrado
#print(sustituir[['precio', 'denominacion', 'precio_desc', 'url' ]]) #'denominacion_desc'
# Restaurar la configuración original después de la impresión si es necesario
#pd.reset_option('display.float_format')

In [48]:
# Filtrar los registros según las condiciones dadas
totales = df[( ((df['precio']!= 0) & (df['precio_desc'].notna())) & (df['precio'] != df['precio_desc']) )] 
# Filtro dentro del estándar
totales = totales[(totales['precio_desc'] >= 250000) & (totales['precio_desc'] <= 500000000)]

# Iterar sobre los registros filtrados
for idx, row in totales.iterrows():
    print(f"Registro {idx}:")
    print(row[['precio', 'precio_desc', 'url']])
    print("\n¿Qué acción te gustaría realizar?")
    print("1. Sustituir el valor manualmente")
    print("2. Sustituir el valor de 'precio' por el de 'precio_desc'")
    print("3. Pasar al siguiente registro")

    # Solicitar la opción al usuario
    option = input("Elige una opción (1, 2, 3): ")

    if option == '1':
        # Sustituir el valor manualmente
        new_value = input("Introduce el nuevo valor para 'precio': ")
        df.at[idx, 'precio'] = float(new_value)
    elif option == '2':
        # Sustituir el valor de 'metros_total' por 'totales_desc'
        df.at[idx, 'precio'] = df.at[idx, 'precio_desc']
    elif option == '3':
        # Pasar al siguiente registro
        continue
    else:
        print("Opción no válida. Pasando al siguiente registro...")
    
    print("\n---\n")

print("Proceso completado.")

Registro 3:
precio                                                                                 336528
precio_desc                                                                         7148520.0
url            https://www.lamudi.com.mx/detalle/41032-73-e8dbd633cb0a-ec3b-1910a3a-8458-704e
Name: 3, dtype: object

¿Qué acción te gustaría realizar?
1. Sustituir el valor manualmente
2. Sustituir el valor de 'precio' por el de 'precio_desc'
3. Pasar al siguiente registro
Registro 66:
precio                                                                                455119
precio_desc                                                                         409919.0
url            https://www.lamudi.com.mx/detalle/41032-73-18fa4ea094b8-b7e-19171ba-a663-7056
Name: 66, dtype: object

¿Qué acción te gustaría realizar?
1. Sustituir el valor manualmente
2. Sustituir el valor de 'precio' por el de 'precio_desc'
3. Pasar al siguiente registro
Registro 67:
precio                                

## b) Denominación MXN

SI EXISTE UNA DENOMINACIÓN QUE NO SEA MXN, SE CONVIERTA EL PRECIO

In [155]:
import datetime

c = CurrencyConverter() # Instancia del convertidor de monedas
def convertir_a_mxn(precio, denominacion):
    if denominacion != 'MXN':
        return int(round(c.convert(precio, denominacion, 'MXN')))
    return precio

def convertir_a_usd(precio, denominacion):
    if denominacion != 'USD':
        return round(c.convert(precio, denominacion, 'USD'), 2)
    return precio

# Obtener la fecha actual
fecha_conversion = datetime.datetime.now().date()

# Aplicar la conversión a cada fila del DataFrame y agregar la fecha de conversión
df['precio_mxn'] = df.apply(lambda row: convertir_a_mxn(row['precio'], row['denominacion']), axis=1)
df['precio_usd'] = df.apply(lambda row: convertir_a_usd(row['precio'], row['denominacion']), axis=1)
#df['denominacion_mxn'] = 'MXN'
#df['denominacion_usd'] = 'USD'
df['fecha_conversion'] = fecha_conversion

#print(df[['precio', 'denominacion', 'precio_mxn', 'denominacion_mxn', 'precio_usd', 'denominacion_usd', 'fecha_conversion']].head())

# Mover las columnas 'precio_mxn' y 'precio_usd' al inicio del DataFrame
#cols = ['precio_mxn', 'precio_usd'] + [col for col in df.columns if col not in ['precio_mxn', 'precio_usd']]
#df = df[cols]

In [156]:
df.head()

,precio,denominacion,propiedad,metros_total,metros_construido,tiempo_de_publicacion,tipo,estacionamientos,recamaras,banos,medio_banos,seguridad_privada,ubicacion,url,descripcion,precio_desc,precio_mxn,precio_usd,fecha_conversion
0,2640000,MXN,CASA VENTA BIOGRAND JURIQUILLA QUERETARO,160,153,2024-03-01 00:00:00,Casa,2,4,2,1,No,"Fraccionamiento San Isidro, Querétaro, Querétaro",https://www.lamudi.com.mx//detalle/41032-73-1ec4b4df5ce1-776e-647beda2-a65a-3948,"Casa en Gran Reserva Juriquilla- Querétaro- OPORTUNIDAD- $2,890,000__3 Recamaras/ 2.5 baños/estudio/160m2 Terreno/ 153m2 Construcción.__Residencia en Gran Reserva Juriquilla. Ubicado en la exclusiva comunidad de Nuevo Juriquilla se encuentra esta casa de impecable diseño. Un maravilloso plano abierto, con techos altos, cocina hermosamente decorada, con cubiertas de granito, grandes gabinetes de madera . Abundante luz natural y grandes ventanales. Tanto la sala de estar como el comedor ofrecen un amplio diseño con jardín para entretener a los invitados. Subiendo las escaleras encontramos la habitación Principal, con amplio vestidor, grandes ventanales que proveen Luz natural , habitaciones secundarias con armarios empotrados. Estacionamiento para 2 autos. EasyBroker ID: EB-PQ0089",NaN,2640000,146849.88,2024-09-09
1,6950000,MXN,VENTA DE CASA EN LA COLONIA CIMATARIO,428,335,2023-11-25 00:00:00,Casa,2,3,3,1,No,"Cimatario, Querétaro, Querétaro",https://www.lamudi.com.mx//detalle/41032-73-a6198fe6bba6-f1a9-d6ff616b-bdb7-3cd2,"VENTA DE CASA EN LA COLONIA CIMATARIO MUY AMPLIA EN UNA EXCELENTE COLONIA DE QUERETARO DENTRO DE FRACCIONAMIENTO PRIVADO CON VIGILANCIA LAS 24 HRS Y MUY CENTRICA CON ACCESO A AVENIDAS PRINCIPALES MUY CERCA DE CENTROS COMERCIALES, ESCUELAS Y HOSPITALES, LA CASA CUENTA CON:UNA SOLA PLANTA3 RECAMARAS AMPLIASCON CLOSETS3 BAÑOS COMPLETOS1 MEDIO BAÑO1 VESTIDOR EN RECAMARA PRINCIPALCOCHERA PARA DOS AUTOSCOCINA EQUIPADACUARTO DE SERVICIOCIRCUITO CERRADOCISTERNAAIRE ACONDICIONADO. EasyBroker ID: EB-OT1308",NaN,6950000,386593.44,2024-09-09
2,2908782,MXN,Casa - Villa Villas del Refugio,141,158,22 ago. 2023,Casa,2,2,2,1,No,"El Refugio, Querétaro, Querétaro",https://www.lamudi.com.mx//detalle/41032-73-b1d6b576904-e9ad-bd805aab-bf0e-3f41,"Casa habitación en fraccionamiento privado en Av. Fray Junìpero Serra, cerca del Centro Comercial más grande de Querétaro. Son dos pisos:En planta baja dos cajones de estacionamiento, acceso, sala comedor, medio baño, cocina, lavandería y jardín.En planta alta tiene recámara principal con baño y vestidor, estudio, recámara secundaria con closet y con baño completo . Se entrega con acabados de lujo, cocina integral con cubierta de cuarzo, baños equipados con canceles de baño, los vestidores tienen madera tratada y están totalmente terminados con acabados de lujo. CELULAR GAMALIEL SORIANO: Ver teléfono Whatsapp*""Debido a la inseguridad, le solicito amablemente, me envíe un WhatsApp con su nombre y datos de su celular, antes de llamarme, para registrarlo y así podré contestarle o comunicarme con usted. Gracias"" Esta información de la Propiedad en Promoción es confidencial y está legalmente protegida y/o puede contener informacion privilegiada. Si usted no es su destinatario o no es alguna persona autorizada para recibirla no deberá utilizar, copiar, revelar, o tomar ninguna acción basada en esta promoción o cualquier otra informacion incluida en este promocional (incluyendo todos los documentos adjuntos). En caso de que este promocional esté dirigido a alguno de nuestros clientes, la información, precio o recomendación contenida está sujeta a cambios de acuerdo a las condiciones regulatorias de DIRECTO BIENES RAICES que resulten aplicables o a los acuerdos comerciales suscritos con el cliente.",NaN,2908782,161800.87,2024-09-09
3,280000,MXN,VENTA DE DEPARTAMENTO TIM 708520,61,61,20 abr. 2023,Departamento,1,2,1,0,No,"Andadores Satélite, Querétaro, Querétaro",https://www.lamudi.com.mx//detalle/41032-73-fa6a90b96237-7270-f886adb-a830-3d89,"Ven

## c) Precio fuera del estandar

IDENTIFICAR Y ELIMINAR PROPIEDADES QUE CUENTEN CON PRECIOS BAJOS y ALTOS AL ESTANDAR

In [62]:
# Eliminar los registros donde el precio es menor al estandar (250,000) y con precio_desc vacío
df = df[(df['precio'] >= 250000)] #& (df['precio_desc'].notna())]
# Eliminar los registros donde el precio es mayor al estandar (500,000,000) y con precio_desc vacío
df = df[(df['precio'] <= 500000000)] #& (df['precio_desc'].notna())] 
# Ordenar valores de manera ascendente
df['precio'].sort_values()

0          400000
1          440450
2          469000
3          500000
4          575000
          ...    
8404     78000000
8405     78000000
8406     92500000
8407    121200000
8408    216910000
Name: precio, Length: 8638, dtype: int64

## D) Segmento o categoría

In [157]:
bins = [0, 500000, 750000, 1000000, 1250000, 1500000, 1750000, 2000000, 2250000, 2500000,
        2750000, 3000000, 3250000, 3500000, 3750000, 4000000, 6000000, 8000000, 12000000,
        14000000, 16000000, 18000000, 22000000, float('inf')]
labels = ["E1", "E2", "E3", "D1", "D2", "D3", "C1", "C2", "C3", "B1", "B2", "B3",
          "A1", "A2", "A3", "S1", "S2", "S3", "L1", "L2", "L3", "L+", "ELITE"]

# Utilizar pd.cut para clasificar los valores
df['categoria'] = pd.cut(df['precio'], bins=bins, labels=labels, right=False)
print(df['categoria'].unique())

['B1', 'S2', 'B2', 'E1', 'E2', ..., 'L3', 'L1', 'S3', 'L+', 'L2']
Length: 23
Categories (23, object): ['E1' < 'E2' < 'E3' < 'D1' ... 'L2' < 'L3' < 'L+' < 'ELITE']


In [124]:
df.head()

,precio,denominacion,propiedad,metros_total,metros_construido,tiempo_de_publicacion,tipo,estacionamientos,recamaras,banos,medio_banos,seguridad_privada,ubicacion,url,descripcion,precio_desc,precio_mxn,precio_usd,fecha_conversion,categoria
0,280000,MXN,venta de departamento tim 708520,61.0,61.0,20-abr-23,Departamento,2,2,1,0,No,"Jurica Misiones, Juriquilla, QuerÃ©taro Arteaga",https://www.lamudi.com.mx/detalle/41032-73-fa6a90b96237-7270-f886adb-a830-3d89,venta de departamento de interas social con salacomedor cocina baao patio de servicio y dos recamaras en un cuarto piso si te interesa este inmueble favor de poner tu correo electronico o envia correo con el inmueble de tu interes a contactar para poder contactarte no se envia informacion por whats app llama al ver telafono no se acepta ningun tipo de credito inmueble invadido escritura en tramite,NaN,280000,15574.99,2024-09-09,E1
1,290000,MXN,casa en venta ubicada en cumbres del lago juriquilla querataro,80.0,80.0,2024-07-29 00:00:00,Casa,1,2,1,0,No,"Cumbres del Lago, Juriquilla, QuerÃ©taro Arteaga",https://www.lamudi.com.mx/detalle/41032-73-a89259ff0118-9510-190fe62-afa7-759b,aaaen venta propiedad de recuperacia3n bancaria gran oportunidad a mas de la mitad del valor del precio comercial el mejor portafolios de remates bancarios la propiedad se encuentra en excelente ubicacia3n te asesoramos durante toda la gestia3n hasta la entrega del inmueble culminamos el proceso en el menor tiempo posible informacia3n legal del bien inmueble garantizamos la entrega de tu nueva casa,NaN,290000,16131.24,2024-09-09,E1
2,328000,MXN,casa en venta villas de santiago queretaro,75.0,90.0,05-abr-24,Casa,2,2,1,1,No,"Villas de Santiago, QuerÃ©taro, QuerÃ©taro Arteaga",https://www.lamudi.com.mx/detalle/41032-73-7802eb158e46-29a5-e1e8474f-aedb-4a9d,esta es una propiedad que se queda3 en garantaa de un cradito hipotecario fallido los pagos son de contado no se acepta ningaon tipo de cradito el tiempo de entrega de la cesion va de 8 a 14 meses que es el tiempo que necesitamos para concluir el proceso a su nombre con mucho gusto le puedo invitar a nuestras oficinas para brindarle mayores informes,NaN,328000,18244.99,2024-09-09,E1
3,336528,MXN,casa en venta cerrito colorado santiago queretaro,80.0,87.0,2024-08-22 00:00:00,Casa,1,3,1,0,No,"Cumbres del Lago, Juriquilla, QuerÃ©taro Arteaga",https://www.lamudi.com.mx/detalle/41032-73-e8dbd633cb0a-ec3b-1910a3a-8458-704e,super oportunidadexcelente propiedad se encuentra en una zona de gran plusvalaarequisitos manimos todo regulado por las autoridades y protegido bajo contrato y notario paoblico garantaa hipotecaria lista para tramitar cesia3n de derechos reciba su propiedad en maximo 12 meses bajo contratocontamos con diversas opciones a tu alcanceagarantiasa garantizamos un 0 de riesgoa garantizamos el tiempo de entregaa tramites ante fedatario publico y autoridades correspondientesagastos juridicos incluidos en nuestros preciosa retroalimentacion constante durante todo el procesoa confidencialidad total de la informacion de nuestros clientescomunicate lo antes posibletu asesor inmobiliariolic hernandezwamever telafono whatsapp 54121381948519 7148520,7148520.0,336528,18719.35,2024-09-09,E1
4,345368,MXN,vendo casa en cerrito colorado alta plusvalia llama y agenda tu asesoria sin costo,70.0,70.0,2023-11-21 00:00:00,Casa,1,2,1,0,No,"Cumbres del Lago, Juriquilla, QuerÃ©taro Arteaga",https://www.lamudi.com.mx/detalle/41032-73-6ae9d93b9d59-6d7d-d37fb8a2-97f5-4bb5,propiedad de recuperacia3n bancariaa no se acepta ningaon tipo de craditoa asnicamente recursos propiosa entrega bajo contrato en un maximo de 14 meses o la devolucia3n de tu inversia3na no se pueden ver por dentro debido al proceso de recuperacia3nel principal beneficio de una propiedad de este tipo es el importante ahorro de hasta un 60 en la adquisicia3n de tu nuevo hogaral recibir tu propiedad esta queda valuada a su valor comercial y no al costo de recuperacia3n lo que significa que en un plazo no may

# 2. Meses transcurridos

## a) Formato de la fecha

In [107]:
# Convertir mes en número
def month_to_int(month):
    months = ['ene', 'feb', 'mar', 'abr', 'may', 'jun', 'jul', 'ago', 'sept', 'oct', 'nov', 'dic']
    return months.index(month.lower()) + 1

# Estandarizar el formato de fecha
def standardize_date(date_str):
    # Asegurarse de que date_str sea una cadena
    date_str = str(date_str)
    
    # Parsear formato "7 oct 2023"
    try:
        day, month, year = date_str.split(' ')
        month_int = month_to_int(month)
        return f"{int(day)} {month} {year}"
    except ValueError:
        pass
    
    # Parsear formato "07/03/2024"
    try:
        day, month, year = date_str.split('/')
        months = ['ene', 'feb', 'mar', 'abr', 'may', 'jun', 'jul', 'ago', 'sept', 'oct', 'nov', 'dic']
        month_str = months[int(month) - 1]
        return f"{int(day)} {month_str} {year}"
    except ValueError:
        pass
    
    # Si no se puede parsear, devolver el string original 
    return date_str
df['tiempo_de_publicacion'] = df['tiempo_de_publicacion'].apply(standardize_date)


## b) Cálculo de meses

Sacar el número de meses transcurridos desde la fecha de su publicación a la actual al ejecutar este script.

In [108]:
# Función para calcular los meses transcurridos
def months_since(date_str, today):
    parts = date_str.split()
    if len(parts) != 3:
        return None  # Return None for invalid date strings
    day, month, year = parts
    try:
        date = datetime(int(year), month_to_int(month), int(day))
        delta = today - date
        return delta.days // 30  # Approximate months
    except ValueError:
        return None  # Return None if date conversion fails

from datetime import datetime  # Import datetime class from datetime module

today = datetime.now()  # Obtener la fecha actual
current_date_str = today.strftime('%Y-%m-%d')

# Calcular los meses transcurridos, columna con la fecha actual en el nombre
new_column_name = f'meses_{current_date_str}'
df[new_column_name] = df['tiempo_de_publicacion'].apply(lambda x: months_since(x, today))
# Filtrar filas con valores None
df = df.dropna(subset=[new_column_name])
#print(df[['tiempo_de_publicacion', new_column_name]])

# 3. Tipo de propiedad

## a) Lista de tipos deseados

In [17]:
# Filtrar los registros según los tipos especificados
tipos_permitidos = 'Casa|Casa en Fraccionamiento|Casa en Condominio|Departamento|Penthouse|Dúplex|Loft|Estudio|Condominio Horizontal'
df = df[df['tipo'].str.contains(tipos_permitidos, na=False)]
df['tipo'].unique()

array(['Casa', 'Casa en Condominio', 'Casa en Fraccionamiento'],
      dtype=object)

## b) Manejo de terrenos

Se buscan los registros que no sean las propiedades si no un solo terreno

In [18]:
# Filtrar los registros según las condiciones dadas
totales =df[df['descripcion'].str.contains('lotes', case=False)]

# MANEJAR PROCESO DE ELIMINACIÓN 
for idx, row in totales.iterrows():
    print(f"Registro {idx}:")
    print(row[['url']])#'descripcion',
    
    print("\n¿Te gustaría eliminar el registro?")
    # Solicitar la opción al usuario
    option = input("Elige una opción (si / no): ").strip().lower()
    
    if option == 'si':
        df.drop(idx, inplace=True)
        print(f"Registro eliminado.\n")
    else:
        print(f"Registro conservado.\n")
        print("\n---\n")
print("Proceso completado.")

Registro 8820:
url    https://www.lamudi.com.mx/detalle/41032-73-dba084047e07-c73a-bddd7b8d-b6fa-386a
Name: 8820, dtype: object

¿Te gustaría eliminar el registro?


Registro conservado.


---

Registro 8822:
url    https://www.lamudi.com.mx/detalle/41032-73-d6cddc7ca082-676c-a81fdfe2-b6bf-3955
Name: 8822, dtype: object

¿Te gustaría eliminar el registro?
Registro conservado.


---

Registro 8823:
url    https://www.lamudi.com.mx/detalle/41032-73-df4589fd6a66-a8c6-efca3927-a263-33f3
Name: 8823, dtype: object

¿Te gustaría eliminar el registro?
Registro conservado.


---

Registro 8959:
url    https://www.lamudi.com.mx/detalle/41032-73-a602f5758367-2479-52a66b4-ac2a-502a
Name: 8959, dtype: object

¿Te gustaría eliminar el registro?
Registro conservado.


---

Proceso completado.


# 4. Metros totales

## a) Metros en descripción

In [25]:
# Función para extraer totales de la descripción
def extraer_totales(descripcion):
    descripcion = descripcion.lower()
    # Buscar patrones de totales
    totales = re.findall(r'\b(\d+[\.,]?\d*)\s*(metros|totales)\b', descripcion)
    total = None
    for match in totales:
        numero, _ = match
        numero = numero.replace(',', '').replace("'", '')
        try:
            total = float(numero)
        except ValueError:
            continue
        if total:
            break
    return total

# Aplicar la función al DataFrame
df['totales_desc'] = df['descripcion'].apply(extraer_totales)

### Comparaciones

#### Metros [0 / 0]

In [26]:
# Filtrar los registros según las condiciones dadas
totales = df[( (df['metros_total'] == 0 ) & (df['totales_desc'].isna() | df['totales_desc']==0) )] 

# Iterar sobre los registros filtrados
for idx, row in totales.iterrows():
    print(f"Registro {idx}:")
    print(row[['metros_total', 'totales_desc', 'url']])
    print("\n¿Qué acción te gustaría realizar?")
    print("1. Sustituir el valor manualmente")
    print("2. Sustituir el valor de 'metros_total' por el de 'totales_desc'")
    print("3. Pasar al siguiente registro")

    # Solicitar la opción al usuario
    option = input("Elige una opción (1, 2, 3): ")

    if option == '1':
        # Sustituir el valor manualmente
        new_value = input("Introduce el nuevo valor para 'metros_total': ")
        df.at[idx, 'metros_total'] = float(new_value)
    elif option == '2':
        # Sustituir el valor de 'metros_total' por 'totales_desc'
        df.at[idx, 'metros_total'] = df.at[idx, 'totales_desc']
    elif option == '3':
        # Pasar al siguiente registro
        continue
    else:
        print("Opción no válida. Pasando al siguiente registro...")
    
    print("\n---\n")

print("Proceso completado.")


Proceso completado.


#### Metros [0 / #]

In [27]:
# Filtrar los registros según las condiciones dadas
totales = df[( (df['metros_total'] == 0 ) & (df['totales_desc'].notna()) )] 

# Iterar sobre los registros filtrados
for idx, row in totales.iterrows():
    print(f"Registro {idx}:")
    print(row[['metros_total', 'totales_desc', 'url']])
    print("\n¿Qué acción te gustaría realizar?")
    print("1. Sustituir el valor manualmente")
    print("2. Sustituir el valor de 'metros_total' por el de 'totales_desc'")
    print("3. Pasar al siguiente registro")

    # Solicitar la opción al usuario
    option = input("Elige una opción (1, 2, 3): ")

    if option == '1':
        # Sustituir el valor manualmente
        new_value = input("Introduce el nuevo valor para 'metros_total': ")
        df.at[idx, 'metros_total'] = float(new_value)
    elif option == '2':
        # Sustituir el valor de 'metros_total' por 'totales_desc'
        df.at[idx, 'metros_total'] = df.at[idx, 'totales_desc']
    elif option == '3':
        # Pasar al siguiente registro
        continue
    else:
        print("Opción no válida. Pasando al siguiente registro...")
    
    print("\n---\n")

print("Proceso completado.")


Proceso completado.


#### Metros [# / #]

In [28]:
# Filtrar los registros según las condiciones dadas
totales = df[( ((df['metros_total']!= 0) & (df['totales_desc'].notna())) & (df['metros_total'] != df['totales_desc']) )] 

# Iterar sobre los registros filtrados
for idx, row in totales.iterrows():
    print(f"Registro {idx}:")
    print(f"metros_total: {row['metros_total']}")
    print(f"totales_desc: {row['totales_desc']}")
    print(f"url: {row['url']}")
    
    print("\n¿Qué acción te gustaría realizar?")
    print("1. Sustituir el valor manualmente")
    print("2. Sustituir el valor de 'metros_total' por el de 'totales_desc'")
    print("3. Pasar al siguiente registro")

    # Solicitar la opción al usuario
    option = input("Elige una opción (1, 2, 3): ")

    if option == '1':
        # Sustituir el valor manualmente
        while True:
            try:
                new_value = float(input("Introduce el nuevo valor para 'metros_total': "))
                df.at[idx, 'metros_total'] = new_value
                break
            except ValueError:
                print("Por favor, introduce un número válido (entero o decimal).")
    elif option == '2':
        # Sustituir el valor de 'metros_total' por 'totales_desc'
        df.at[idx, 'metros_total'] = df.at[idx, 'totales_desc']
    elif option == '3':
        # Pasar al siguiente registro
        continue
    else:
        print("Opción no válida. Pasando al siguiente registro...")
    
    print("\n---\n")

print("Proceso completado.")


Registro 593:
metros_total: 123
totales_desc: 0.0
url: https://www.lamudi.com.mx/detalle/41032-73-85bc36bf8eaf-6efb-327a921b-859e-4c09

¿Qué acción te gustaría realizar?
1. Sustituir el valor manualmente
2. Sustituir el valor de 'metros_total' por el de 'totales_desc'
3. Pasar al siguiente registro


Registro 849:
metros_total: 200
totales_desc: 14.0
url: https://www.lamudi.com.mx/detalle/41032-73-9a66a6c61eb4-78df-71c0c7b7-bc27-3f1c

¿Qué acción te gustaría realizar?
1. Sustituir el valor manualmente
2. Sustituir el valor de 'metros_total' por el de 'totales_desc'
3. Pasar al siguiente registro


C:\Users\yoe11\AppData\Local\Temp\ipykernel_3552\3639181133.py:24: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '200.2' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df.at[idx, 'metros_total'] = new_value



---

Registro 866:
metros_total: 85
totales_desc: 85.8
url: https://www.lamudi.com.mx/detalle/41032-73-b56f96c0511a-c34-d92735bc-a2b1-370e

¿Qué acción te gustaría realizar?
1. Sustituir el valor manualmente
2. Sustituir el valor de 'metros_total' por el de 'totales_desc'
3. Pasar al siguiente registro

---

Registro 1293:
metros_total: 105
totales_desc: 15.0
url: https://www.lamudi.com.mx/detalle/41032-73-98657fa95fe1-5ebb-c94a7aac-abf1-3a73

¿Qué acción te gustaría realizar?
1. Sustituir el valor manualmente
2. Sustituir el valor de 'metros_total' por el de 'totales_desc'
3. Pasar al siguiente registro
Registro 5119:
metros_total: 215
totales_desc: 3.8
url: https://www.lamudi.com.mx/detalle/41032-73-f4b01f755545-aa0d-5a5bef84-a121-3457

¿Qué acción te gustaría realizar?
1. Sustituir el valor manualmente
2. Sustituir el valor de 'metros_total' por el de 'totales_desc'
3. Pasar al siguiente registro

---

Registro 5120:
metros_total: 215
totales_desc: 3.8
url: https://www.lamudi.com.m

## b) Manejo por rangos de UMA

Rangos
* 118 UMAS = 40m2
* 118.1-200 = 50m2
* 200.1-350 = 71m2
* 350.1-750 = 102m2
* 750.1-1500 = 156m2

### DF cuando ambas columnas son 0 o vacíos

In [29]:
# Rangos de UMAs y sus equivalencias en m²
umas_to_m2 = [
    (118, 40),
    (200, 50),
    (350, 71),
    (750, 102),
    (1500, 156)
]
valor_uma = 108.57

# Convertir MXN a UMAs
def precio_a_umas(precio, valor_uma):
    return precio / valor_uma

# Asignar m² basado en el precio en UMAs
def asignar_m2(precio_umas):
    for upper_bound, m2 in umas_to_m2:
        if precio_umas <= upper_bound:
            return m2
    return None  # En caso de que el precio en UMAs sea mayor al máximo definido

# Actualizar los registros con 'metros_total' igual a 0 y 'totales_desc' es NaN
def actualizar_metros_total(row):
    if row['metros_total'] == 0 and pd.isna(row['totales_desc']):
        precio_umas = precio_a_umas(row['precio'], valor_uma)
        row['metros_total'] = asignar_m2(precio_umas)
    return row

df = df.apply(actualizar_metros_total, axis=1)
print(df[['metros_total', 'totales_desc', 'url']])

      metros_total  totales_desc  \
6            100.0           NaN   
13           350.0           NaN   
18           500.0           NaN   
31           483.0           NaN   
37           277.0           NaN   
...            ...           ...   
9285         900.0           NaN   
9317         634.0           NaN   
9321        1047.0           NaN   
9327         750.0           NaN   
9332        1047.0           NaN   

                                                                                  url  
6     https://www.lamudi.com.mx/detalle/41032-73-7470e7c0f69b-8f03-1adcb58d-9a14-405d  
13    https://www.lamudi.com.mx/detalle/41032-73-bd5a86939b0e-2cb6-83bc8753-80aa-49c5  
18     https://www.lamudi.com.mx/detalle/41032-73-1db2d29a0f95-16c-34653316-be19-4b01  
31     https://www.lamudi.com.mx/detalle/41032-73-93d194dc33d-d035-bf07501c-825e-4a4a  
37    https://www.lamudi.com.mx/detalle/41032-73-27915b7692e9-3bc0-67bbfd01-9453-4608  
...                                    

# 5. M2 construidos

## a) M-construidos en descripción

In [30]:
# Función para extraer totales de la descripción
def extraer_totales(descripcion):
    descripcion = descripcion.lower()
    # Buscar patrones de totales
    totales = re.findall(r'\b(\d+[\.,]?\d*)\s*(metros|construidos)\b', descripcion)
    total = None
    for match in totales:
        numero, _ = match
        numero = numero.replace(',', '').replace("'", '')
        try:
            total = float(numero)
        except ValueError:
            continue
        if total:
            break
    return total

# Aplicar la función al DataFrame
df['construidos_desc'] = df['descripcion'].apply(extraer_totales)

### Comparaciones

#### Metros [0 / 0]

In [31]:
# Filtrar los registros según las condiciones dadas
totales = df[( (df['metros_construido'] == 0 ) & (df['construidos_desc'].isna() | df['construidos_desc']==0) )] 

# Iterar sobre los registros filtrados
for idx, row in totales.iterrows():
    print(f"Registro {idx}:")
    print(row[['metros_construido', 'construidos_desc', 'url']])
    print("\n¿Qué acción te gustaría realizar?")
    print("1. Sustituir el valor manualmente")
    print("2. Sustituir el valor de 'metros_construido' por el de 'construidos_desc'")
    print("3. Pasar al siguiente registro")

    # Solicitar la opción al usuario
    option = input("Elige una opción (1, 2, 3): ")

    if option == '1':
        # Sustituir el valor manualmente
        new_value = input("Introduce el nuevo valor para 'metros_construido': ")
        df.at[idx, 'metros_construido'] = float(new_value)
    elif option == '2':
        # Sustituir el valor de 'metros_construido' por 'construidos_desc'
        df.at[idx, 'metros_construido'] = df.at[idx, 'construidos_desc']
    elif option == '3':
        # Pasar al siguiente registro
        continue
    else:
        print("Opción no válida. Pasando al siguiente registro...")
    
    print("\n---\n")

print("Proceso completado.")


Proceso completado.


#### Metros [0 / #]

In [32]:
# Filtrar los registros según las condiciones dadas
totales = df[( (df['metros_construido'] == 0 ) & (df['construidos_desc'].notna()) )] 

# Iterar sobre los registros filtrados
for idx, row in totales.iterrows():
    print(f"Registro {idx}:")
    print(row[['metros_construido', 'construidos_desc', 'url']])
    print("\n¿Qué acción te gustaría realizar?")
    print("1. Sustituir el valor manualmente")
    print("2. Sustituir el valor de 'metros_construido' por el de 'construidos_desc'")
    print("3. Pasar al siguiente registro")

    # Solicitar la opción al usuario
    option = input("Elige una opción (1, 2, 3): ")

    if option == '1':
        # Sustituir el valor manualmente
        new_value = input("Introduce el nuevo valor para 'metros_construido': ")
        df.at[idx, 'metros_construido'] = float(new_value)
    elif option == '2':
        # Sustituir el valor de 'metros_construido' por 'construidos_desc'
        df.at[idx, 'metros_construido'] = df.at[idx, 'construidos_desc']
    elif option == '3':
        # Pasar al siguiente registro
        continue
    else:
        print("Opción no válida. Pasando al siguiente registro...")
    
    print("\n---\n")

print("Proceso completado.")


Proceso completado.


#### Metros [# / #]

In [33]:
# Filtrar los registros según las condiciones dadas
totales = df[( ((df['metros_construido']!= 0) & (df['construidos_desc'].notna())) & (df['metros_construido'] != df['construidos_desc']) )] 

# Iterar sobre los registros filtrados
for idx, row in totales.iterrows():
    print(f"Registro {idx}:")
    print(row[['metros_construido', 'construidos_desc', 'url']])
    print("\n¿Qué acción te gustaría realizar?")
    print("1. Sustituir el valor manualmente")
    print("2. Sustituir el valor de 'metros_construido' por el de 'construidos_desc'")
    print("3. Pasar al siguiente registro")

    # Solicitar la opción al usuario
    option = input("Elige una opción (1, 2, 3): ")

    if option == '1':
        # Sustituir el valor manualmente
        new_value = input("Introduce el nuevo valor para 'metros_construido': ")
        try:
            df.at[idx, 'metros_construido'] = float(new_value)
        except ValueError:
            print("Valor no válido. Debe ser un número. Pasando al siguiente registro...")
    elif option == '2':
        # Sustituir el valor de 'metros_construido' por 'construidos_desc'
        df.at[idx, 'metros_construido'] = df.at[idx, 'construidos_desc']
    elif option == '3':
        # Pasar al siguiente registro
        continue
    else:
        print("Opción no válida. Pasando al siguiente registro...")
    
    print("\n---\n")

print("Proceso completado.")


Registro 593:
metros_construido                                                                                123
construidos_desc                                                                                 0.0
url                  https://www.lamudi.com.mx/detalle/41032-73-85bc36bf8eaf-6efb-327a921b-859e-4c09
Name: 593, dtype: object

¿Qué acción te gustaría realizar?
1. Sustituir el valor manualmente
2. Sustituir el valor de 'metros_construido' por el de 'construidos_desc'
3. Pasar al siguiente registro
Registro 849:
metros_construido                                                                                200
construidos_desc                                                                                14.0
url                  https://www.lamudi.com.mx/detalle/41032-73-9a66a6c61eb4-78df-71c0c7b7-bc27-3f1c
Name: 849, dtype: object

¿Qué acción te gustaría realizar?
1. Sustituir el valor manualmente
2. Sustituir el valor de 'metros_construido' por el de 'construidos_desc'

# 6. Precio / m2

## M2 Totales (terreno)

In [158]:
# Calcular precio por metro cuadrado del terreno
df['precio_m2_terreno'] = df['precio']/df['metros_total']

## M2 Construidos

In [159]:
# Calcular precio por metro cuadrado de la cosntrucción
df['precio_m2_construido'] = df['precio']/df['metros_construido']

# 7. Estacionamientos

## a) Estacionamientos descritos

In [12]:
# Función para extraer estacionamientos de la descripción
def extraer_coches(descripcion):
    descripcion = descripcion.lower()
    # Buscar patrones de totales
    totales = re.findall(r'\b(\d+[\.,]?\d*)\s*(|cochera|garage|carros|carro|coche|coches|autos|automóviles|automóvil|auto|lugares de estacionamiento)\b', descripcion)
    total = None
    for match in totales:
        numero, _ = match
        numero = numero.replace(',', '').replace("'", '').replace('.','')
        try:
            total = float(numero)
        except ValueError:
            continue
        if total:
            break
    return total

# Aplicar la función al DataFrame
df['estacionamiento_desc'] = df['descripcion'].apply(extraer_coches)

### Estacionamientos [0 / 0]

In [38]:
# Filtrar los registros según las condiciones dadas
totales = df[( (df['estacionamientos']==0) & (df['estacionamiento_desc'].isna()) )] 

# Iterar sobre los registros filtrados
for idx, row in totales.iterrows():
    print(f"Registro {idx}:")
    print(row[['url']])
    print("\n¿Qué acción te gustaría realizar?")
    print("1. Sustituir el valor manualmente")
    print("2. Pasar al siguiente registro")

    # Solicitar la opción al usuario
    option = input("Elige una opción (1 | 2): ")

    if option == '1':
        # Sustituir el valor manualmente
        new_value = input("Introduce el nuevo valor para 'estacionamientos': ")
        df.at[idx, 'estacionamientos'] = float(new_value)    
    elif option == '2':
        # Pasar al siguiente registro
        continue
    else:
        print("Opción no válida. Pasando al siguiente registro...")
    
    print("\n---\n")

print("Proceso completado.")


Proceso completado.


### Estacionamientos [0 / #]

In [39]:
# Filtrar los registros según las condiciones dadas
totales = df[( (df['estacionamientos']==0) & (df['estacionamiento_desc'].notna()) )] 

# Iterar sobre los registros filtrados
for idx, row in totales.iterrows():
    print(f"Registro {idx}:")
    print(row[['estacionamientos', 'estacionamiento_desc', 'url']])
    print("\n¿Qué acción te gustaría realizar?")
    print("1. Sustituir el valor manualmente")
    print("2. Sustituir el valor de 'estacionamientos' por el de 'estacionamiento_desc'")
    print("3. Pasar al siguiente registro")

    # Solicitar la opción al usuario
    option = input("Elige una opción (1, 2, 3): ")

    if option == '1':
        # Sustituir el valor manualmente
        new_value = input("Introduce el nuevo valor para 'estacionamientos': ")
        df.at[idx, 'estacionamientos'] = float(new_value)
    elif option == '2':
        # Sustituir el valor de 'metros_total' por 'totales_desc'
        df.at[idx, 'estacionamientos'] = df.at[idx, 'estacionamiento_desc']
    elif option == '3':
        # Pasar al siguiente registro
        continue
    else:
        print("Opción no válida. Pasando al siguiente registro...")
    
    print("\n---\n")

print("Proceso completado.")


Proceso completado.


### Estacionamientos [# / #]

In [40]:
# Filtrar los registros según las condiciones dadas
totales = df[( ((df['estacionamientos']!=0) & (df['estacionamiento_desc'].notna())) & (df['estacionamientos'] != df['estacionamiento_desc']) )] 

# Iterar sobre los registros filtrados
for idx, row in totales.iterrows():
    print(f"Registro {idx}:")
    print(row[['estacionamientos', 'estacionamiento_desc', 'url']])
    print("\n¿Qué acción te gustaría realizar?")
    print("1. Sustituir el valor manualmente")
    print("2. Sustituir el valor de 'estacionamientos' por el de 'estacionamiento_desc'")
    print("3. Pasar al siguiente registro")

    # Solicitar la opción al usuario
    option = input("Elige una opción (1, 2, 3): ")

    if option == '1':
        # Sustituir el valor manualmente
        new_value = input("Introduce el nuevo valor para 'estacionamientos': ")
        df.at[idx, 'estacionamientos'] = float(new_value)
    elif option == '2':
        # Sustituir el valor de 'metros_total' por 'totales_desc'
        df.at[idx, 'estacionamientos'] = df.at[idx, 'estacionamiento_desc']
    elif option == '3':
        # Pasar al siguiente registro
        continue
    else:
        print("Opción no válida. Pasando al siguiente registro...")
    
    print("\n---\n")

print("Proceso completado.")


Registro 6:
estacionamientos                                                                                      2
estacionamiento_desc                                                                                3.0
url                     https://www.lamudi.com.mx/detalle/41032-73-7470e7c0f69b-8f03-1adcb58d-9a14-405d
Name: 6, dtype: object

¿Qué acción te gustaría realizar?
1. Sustituir el valor manualmente
2. Sustituir el valor de 'estacionamientos' por el de 'estacionamiento_desc'
3. Pasar al siguiente registro
Registro 13:
estacionamientos                                                                                      2
estacionamiento_desc                                                                                4.0
url                     https://www.lamudi.com.mx/detalle/41032-73-bd5a86939b0e-2cb6-83bc8753-80aa-49c5
Name: 13, dtype: object

¿Qué acción te gustaría realizar?
1. Sustituir el valor manualmente
2. Sustituir el valor de 'estacionamientos' por el de 'est

## b) Mantener rango de estacionamientos < 5 (mínimo 1)

In [ ]:
df_filtrado = df[df['estacionamientos'] <= 5]
df_filtrado[['estacionamientos','estacionamiento_desc','url']]

In [ ]:
df = df[df['estacionamientos'] <= 5]

# 8. Recámaras

## a) Recámaras en descripción

In [59]:
# Función para extraer cantidad de recamaras en descripción
def extraer_recamaras(descripcion):
    descripcion = descripcion.lower()
    # Buscar patrones de totales
    totales = re.findall(r'\b(\d+[\.,]?\d*)\s*(recamara|recamaras|habitaciones|dormitorio|dormitorios)\b', descripcion)
    total = None
    for match in totales:
        numero, _ = match
        numero = numero.replace(',', '').replace("'", '')
        try:
            total = float(numero)
        except ValueError:
            continue
        if total:
            break
    return total

# Aplicar la función al DataFrame
df['recamaras_desc'] = df['descripcion'].apply(extraer_recamaras)

### Comparaciones

#### Recámaras [0 / 0]

In [53]:
# Filtrar los registros según las condiciones dadas
totales = df[ (df['recamaras'] == 0 ) & (df['recamaras_desc'].isna()) ] 

# Iterar sobre los registros filtrados
for idx, row in totales.iterrows():
    print(f"Registro {idx}:")
    print(row[['url']])
    print("\n¿Qué acción te gustaría realizar?")
    print("1. Sustituir el valor manualmente")
    print("2. Pasar al siguiente registro")

    # Solicitar la opción al usuario
    option = input("Elige una opción (1 | 2): ")

    if option == '1':
        # Sustituir el valor manualmente
        new_value = input("Introduce el nuevo valor para 'recamaras': ")
        df.at[idx, 'recamaras'] = float(new_value)
    elif option == '2':
        # Pasar al siguiente registro
        continue
    else:
        print("Opción no válida. Pasando al siguiente registro...")
    
    print("\n---\n")

print("Proceso completado.")

Proceso completado.


#### Recámaras [0 / #]

In [54]:
# Filtrar los registros según las condiciones dadas
totales = df[( (df['recamaras']==0) & (df['recamaras_desc'].notna()) )] 

# Iterar sobre los registros filtrados
for idx, row in totales.iterrows():
    print(f"Registro {idx}:")
    print(row[['recamaras', 'recamaras_desc', 'url']])
    print("\n¿Qué acción te gustaría realizar?")
    print("1. Sustituir el valor manualmente")
    print("2. Sustituir el valor de 'recamaras' por el de 'recamaras_desc'")
    print("3. Pasar al siguiente registro")

    # Solicitar la opción al usuario
    option = input("Elige una opción (1, 2, 3): ")

    if option == '1':
        # Sustituir el valor manualmente
        new_value = input("Introduce el nuevo valor para 'recamaras': ")
        df.at[idx, 'recamaras'] = float(new_value)
    elif option == '2':
        # Sustituir el valor de 'metros_total' por 'totales_desc'
        df.at[idx, 'recamaras'] = df.at[idx, 'recamaras_desc']
    elif option == '3':
        # Pasar al siguiente registro
        continue
    else:
        print("Opción no válida. Pasando al siguiente registro...")
    
    print("\n---\n")

print("Proceso completado.")


Proceso completado.


#### Recámaras [# / #]

In [ ]:
# Filtrar los registros según las condiciones dadas
totales = df[( (df['recamaras']!=0) & (df['recamaras_desc'].notna()) ) & (df['recamaras'] != df['recamaras_desc'])] 

# Iterar sobre los registros filtrados
for idx, row in totales.iterrows():
    print(f"Registro {idx}:")
    print(row[['recamaras', 'recamaras_desc', 'url']])
    print("\n¿Qué acción te gustaría realizar?")
    print("1. Sustituir el valor manualmente")
    print("2. Sustituir el valor de 'recamaras' por el de 'recamaras_desc'")
    print("3. Pasar al siguiente registro")

    # Solicitar la opción al usuario
    option = input("Elige una opción (1, 2, 3): ")

    if option == '1':
        # Sustituir el valor manualmente
        new_value = input("Introduce el nuevo valor para 'recamaras': ")
        df.at[idx, 'recamaras'] = float(new_value)
    elif option == '2':
        # Sustituir el valor de 'metros_total' por 'totales_desc'
        df.at[idx, 'recamaras'] = df.at[idx, 'recamaras_desc']
    elif option == '3':
        # Pasar al siguiente registro
        continue
    else:
        print("Opción no válida. Pasando al siguiente registro...")
    
    print("\n---\n")

print("Proceso completado.")


## b) Mantener rango de recámaras <= 7 

In [ ]:
df['recamaras'].unique()

In [ ]:
dentro = df[(df['recamaras'] <= 7)] 
print(dentro[['recamaras','recamaras_desc','url']])

# 9. Baños

## a) Baños en descripción

Manejando tanto los completos como los medios

In [160]:
# Extraer la cantidad de baños y medios baños en la descripción
def extraer_baños(descripcion):
    descripcion = descripcion.lower()
    
    # Buscar patrones de baños completos
    completos = re.findall(r'\b(\d+[\.,]?\d*)\s*baños?\b', descripcion)
    total_completos = sum([float(match.replace(',', '').replace("'", '')) for match in completos])
    
    # Buscar patrones de medios baños (1/2 baño)
    medios = re.findall(r'\b(\d+[\.,]?\d*)\s*medios?\s*baños?\b', descripcion)
    total_medios = sum([float(match.replace(',', '').replace("'", '')) for match in medios])
    
    # Retornar como un diccionario o una tupla
    return {'completos': total_completos, 'medios': total_medios}

# Aplicar la función al DataFrame
df['baños_desc'] = df['descripcion'].apply(extraer_baños)

# Si deseas separar en dos columnas distintas:
df['baños_completos'] = df['baños_desc'].apply(lambda x: x['completos'] if x else 0)
df['medios_baños'] = df['baños_desc'].apply(lambda x: x['medios'] if x else 0)

### Baños

#### [0 / 0]

Poner como mínimo para el registro 1 en baños, pero si cuenta con medio baño en descripción que se le asigne ese valor.

In [14]:
# Filtrar los registros según las condiciones dadas
totales = df[(df['banos'] == 0) & (df['baños_completos'].isna())]

# Iterar sobre los registros filtrados
for idx, row in totales.iterrows():
    print(f"Registro {idx}:")
    print(row[['url']])
    print("\n¿Qué acción te gustaría realizar?")
    print("1. Sustituir el valor manualmente")
    #print("2. Asignar 1 como mínimo y pasar al siguiente registro")
    print("2. Pasar al siguiente registro")

    # Solicitar la opción al usuario
    option = input("Elige una opción (1 | 2): ")

    if option == '1':
        # Sustituir el valor manualmente
        new_value = input("Introduce el nuevo valor para 'banos': ")
        df.at[idx, 'banos'] = float(new_value)
    #elif option == '2':
        # Asignar valor de 1 como mínimo
        #df.at[idx, 'banos'] = 1
        #continue  # Pasar al siguiente registro
    else:
        continue  # Pasar al siguiente registro

    print("\n---\n")

print("Proceso completado.")

Proceso completado.


#### [0 / #]

In [63]:
# Filtrar los registros según las condiciones dadas
totales = df[( (df['banos']==0) & (df['baños_completos'].notna()) )] 

# Iterar sobre los registros filtrados
for idx, row in totales.iterrows():
    print(f"Registro {idx}:")
    print(row[['banos']])
    print(row[['baños_completos']])
    print(row[['url']])
    print("\n¿Qué acción te gustaría realizar?")
    print("1. Sustituir el valor manualmente")
    print("2. Sustituir el valor de 'banos' por el de 'baños_completos'")
    #print("3. Asignar 1 como mínimo y pasar al siguiente registro")
    print("3. Pasar al siguiente registro")

    # Solicitar la opción al usuario
    option = input("Elige una opción (1 | 2 | 3): ")

    if option == '1':
        # Sustituir el valor manualmente
        new_value = input("Introduce el nuevo valor para 'banos': ")
        df.at[idx, 'banos'] = float(new_value)
    elif option == '2':
        # Sustituir el valor de 'banos' por 'baños_completos'
        df.at[idx, 'banos'] = df.at[idx, 'baños_completos']
    #elif option == '3':
        # Asignar valor de 1 como mínimo
        #df.at[idx, 'banos'] = 1
        #continue  # Pasar al siguiente registro
    else:
        continue  # Pasar al siguiente registro

    print("\n---\n")

print("Proceso completado.")

Registro 236:
banos    0.0
Name: 236, dtype: object
baños_completos    0.0
Name: 236, dtype: object
url    https://www.lamudi.com.mx/detalle/41032-73-a3ee8bf8fc9d-524a-1f3db509-83cd-4b7b
Name: 236, dtype: object

¿Qué acción te gustaría realizar?
1. Sustituir el valor manualmente
2. Sustituir el valor de 'banos' por el de 'baños_completos'
3. Pasar al siguiente registro


KeyboardInterrupt: Interrupted by user

#### [# / #]

In [58]:
# Filtrar los registros según las condiciones dadas
totales = df[( ((df['banos']!=0) & (df['baños_completos'].notna())) & (df['banos'] != df['baños_completos']))] 

# Iterar sobre los registros filtrados
for idx, row in totales.iterrows():
    print(f"Registro {idx}:")
    print(row[['banos']])
    print(row[['baños_completos']])    
    print(row[['url']])
    
    print("\n¿Qué acción te gustaría realizar?")
    print("1. Sustituir el valor manualmente")
    print("2. Sustituir el valor de 'banos' por el de 'baños_completos'")
    #print("3. Asignar 1 como mínimo y pasar al siguiente registro")
    print("3. Pasar al siguiente registro")

    # Solicitar la opción al usuario
    option = input("Elige una opción (1 | 2 | 3): ")

    if option == '1':
        # Sustituir el valor manualmente
        new_value = input("Introduce el nuevo valor para 'banos': ")
        df.at[idx, 'banos'] = float(new_value)
    elif option == '2':
        # Sustituir el valor de 'banos' por 'baños_completos'
        df.at[idx, 'banos'] = df.at[idx, 'baños_completos']
    #elif option == '3':
        # Asignar valor de 1 como mínimo
        #df.at[idx, 'banos'] = 1
        #continue  # Pasar al siguiente registro
    else:
        continue  # Pasar al siguiente registro

    print("\n---\n")

print("Proceso completado.")

Registro 31:
banos    2
Name: 31, dtype: object
baños_completos    5.0
Name: 31, dtype: object
url    https://www.lamudi.com.mx/detalle/41032-73-93d194dc33d-d035-bf07501c-825e-4a4a
Name: 31, dtype: object

¿Qué acción te gustaría realizar?
1. Sustituir el valor manualmente
2. Sustituir el valor de 'banos' por el de 'baños_completos'
3. Pasar al siguiente registro
Registro 37:
banos    3
Name: 37, dtype: object
baños_completos    0.0
Name: 37, dtype: object
url    https://www.lamudi.com.mx/detalle/41032-73-27915b7692e9-3bc0-67bbfd01-9453-4608
Name: 37, dtype: object

¿Qué acción te gustaría realizar?
1. Sustituir el valor manualmente
2. Sustituir el valor de 'banos' por el de 'baños_completos'
3. Pasar al siguiente registro
Registro 65:
banos    1
Name: 65, dtype: object
baños_completos    0.0
Name: 65, dtype: object
url    https://www.lamudi.com.mx/detalle/41032-73-c7136c4c8f36-cb2f-fd601220-8b1e-4305
Name: 65, dtype: object

¿Qué acción te gustaría realizar?
1. Sustituir el valor manu

C:\Users\yoe11\AppData\Local\Temp\ipykernel_10972\1369790780.py:26: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '3.5' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df.at[idx, 'banos'] = df.at[idx, 'baños_completos']



---

Registro 6993:
banos    1
Name: 6993, dtype: object
baños_completos    0.0
Name: 6993, dtype: object
url    https://www.lamudi.com.mx/detalle/41032-73-eb0c79af90fc-430-e6bbff06-a0f7-33ad
Name: 6993, dtype: object

¿Qué acción te gustaría realizar?
1. Sustituir el valor manualmente
2. Sustituir el valor de 'banos' por el de 'baños_completos'
3. Pasar al siguiente registro
Registro 7019:
banos    1
Name: 7019, dtype: object
baños_completos    0.0
Name: 7019, dtype: object
url    https://www.lamudi.com.mx/detalle/41032-73-50efc0ce8b46-81d9-289506e0-957b-402a
Name: 7019, dtype: object

¿Qué acción te gustaría realizar?
1. Sustituir el valor manualmente
2. Sustituir el valor de 'banos' por el de 'baños_completos'
3. Pasar al siguiente registro
Registro 7020:
banos    1
Name: 7020, dtype: object
baños_completos    0.0
Name: 7020, dtype: object
url    https://www.lamudi.com.mx/detalle/41032-73-b89ad56639c5-4da8-7f8d22f8-99a6-3892
Name: 7020, dtype: object

¿Qué acción te gustaría realiz

### Medio baños

#### [0 / 0]

Poner como mínimo para el registro 1 en baños, pero si cuenta con valor en descripción que se le asigne ese valor.

In [60]:
# Filtrar los registros según las condiciones dadas
totales = df[(df['medio_baños'] == 0) & (df['medios_baños'].isna())]

# Iterar sobre los registros filtrados
for idx, row in totales.iterrows():
    print(f"Registro {idx}:")
    print(row[['url']])
    print("\n¿Qué acción te gustaría realizar?")
    print("1. Sustituir el valor manualmente")
    #print("2. Asignar 1 como mínimo y pasar al siguiente registro")
    print("2. Pasar al siguiente registro")

    # Solicitar la opción al usuario
    option = input("Elige una opción (1 | 2): ")

    if option == '1':
        # Sustituir el valor manualmente
        new_value = input("Introduce el nuevo valor para 'banos': ")
        df.at[idx, 'banos'] = float(new_value)
    #elif option == '2':
        # Asignar valor de 1 como mínimo
        #df.at[idx, 'banos'] = 1
        #continue  # Pasar al siguiente registro
    else:
        continue  # Pasar al siguiente registro

    print("\n---\n")

print("Proceso completado.")

KeyError: 'medio_baños'

#### [0 / #]

In [ ]:
# Filtrar los registros según las condiciones dadas
totales = df[( (df['medio_baños']==0) & (df['medios_baños'].notna()) )] 

# Iterar sobre los registros filtrados
for idx, row in totales.iterrows():
    print(f"Registro {idx}:")
    print(row[['url']])
    print("\n¿Qué acción te gustaría realizar?")
    print("1. Sustituir el valor manualmente")
    print("2. Sustituir el valor de 'medio_baños' por el de 'medios_baños'")
    #print("3. Asignar 1 como mínimo y pasar al siguiente registro")
    print("3. Pasar al siguiente registro")

    # Solicitar la opción al usuario
    option = input("Elige una opción (1 | 2 | 3): ")

    if option == '1':
        # Sustituir el valor manualmente
        new_value = input("Introduce el nuevo valor para 'medio_baños': ")
        df.at[idx, 'medio_baños'] = float(new_value)
    elif option == '2':
        # Sustituir el valor de 'medio_baños' por 'medios_baños'
        df.at[idx, 'medio_baños'] = df.at[idx, 'medios_baños']
    #elif option == '3':
        # Asignar valor de 1 como mínimo
        #df.at[idx, 'medio_baños'] = 1
        #continue  # Pasar al siguiente registro
    else:
        continue  # Pasar al siguiente registro

    print("\n---\n")

print("Proceso completado.")

#### [# / #]

In [ ]:
# Filtrar los registros según las condiciones dadas
totales = df[( ((df['medio_baños']!=0) & (df['medios_baños'].notna())) & (df['medio_baños'] != df['medios_baños']))] 

# Iterar sobre los registros filtrados
for idx, row in totales.iterrows():
    print(f"Registro {idx}:")
    print(row[['url']])
    print("\n¿Qué acción te gustaría realizar?")
    print("1. Sustituir el valor manualmente")
    print("2. Sustituir el valor de 'medio_baños' por el de 'medios_baños'")
    #print("3. Asignar 1 como mínimo y pasar al siguiente registro")
    print("3. Pasar al siguiente registro")

    # Solicitar la opción al usuario
    option = input("Elige una opción (1 | 2 | 3): ")

    if option == '1':
        # Sustituir el valor manualmente
        new_value = input("Introduce el nuevo valor para 'medio_baños': ")
        df.at[idx, 'medio_baños'] = float(new_value)
    elif option == '2':
        # Sustituir el valor de 'medio_baños' por 'medios_baños'
        df.at[idx, 'medio_baños'] = df.at[idx, 'medios_baños']
    #elif option == '3':
        # Asignar valor de 1 como mínimo
        #df.at[idx, 'medio_baños'] = 1
        #continue  # Pasar al siguiente registro
    else:
        continue  # Pasar al siguiente registro

    print("\n---\n")

print("Proceso completado.")

### B) Total de baños

In [68]:
df.columns

Index(['precio', 'denominacion', 'propiedad', 'metros_total',
       'metros_construido', 'tiempo_de_publicacion', 'tipo',
       'estacionamientos', 'recamaras', 'banos', 'medio_banos',
       'seguridad_privada', 'ubicacion', 'url', 'descripcion', 'precio_desc',
       'precio_mxn', 'precio_usd', 'fecha_conversion', 'categoria',
       'precio_m2_terreno', 'precio_m2_construido', 'baños_desc',
       'baños_completos', 'medios_baños'],
      dtype='object')

In [128]:
df.head()

,precio,denominacion,propiedad,metros_total,metros_construido,tiempo_de_publicacion,tipo,estacionamientos,recamaras,banos,...,precio_desc,precio_mxn,precio_usd,fecha_conversion,categoria,precio_m2_terreno,precio_m2_construido,baños_desc,baños_completos,medios_baños
0,280000,MXN,venta de departamento tim 708520,61.0,61.0,20-abr-23,Departamento,2,2,1,...,NaN,280000,15574.99,2024-09-09,E1,4590.163934,4590.163934,"{'completos': 0, 'medios': 0}",0,0
1,290000,MXN,casa en venta ubicada en cumbres del lago juriquilla querataro,80.0,80.0,2024-07-29 00:00:00,Casa,1,2,1,...,NaN,290000,16131.24,2024-09-09,E1,3625.000000,3625.000000,"{'completos': 0, 'medios': 0}",0,0
2,328000,MXN,casa en venta villas de santiago queretaro,75.0,90.0,05-abr-24,Casa,2,2,1,...,NaN,328000,18244.99,2024-09-09,E1,4373.333333,3644.444444,"{'completos': 0, 'medios': 0}",0,0
3,336528,MXN,casa en venta cerrito colorado santiago queretaro,80.0,87.0,2024-08-22 00:00:00,Casa,1,3,1,...,7148520.0,336528,18719.35,2024-09-09,E1,4206.600000,3868.137931,"{'completos': 0, 'medios': 0}",0,0
4,345368,MXN,vendo casa en cerrito colorado alta plusvalia llama y agenda tu asesoria sin costo,70.0,70.0,2023-11-21 00:00:00,Casa,1,2,1,...,NaN,345368,19211.08,2024-09-09,E1,4933.828571,4933.828571,"{'completos': 0, 'medios': 0}",0,0


In [161]:
# Make the medio_baños and banos sum
# 'medio_baños', divididos entre 2 para representar medios
df['bano_total'] = df['banos'] + (df['medio_banos'] / 2)
df['bano_total'] = df['bano_total'].round(1)  # Round the total, not just 'banos'
print(df[['banos', 'medio_banos', 'bano_total']].head())

   banos  medio_banos  bano_total
0      2            1         2.5
1      3            1         3.5
2      2            1         2.5
3      1            0         1.0
4      3            0         3.0


# 10. Ubicación

## a) Desde descripción
#### Todavía en espera de estructura de código

## b) Separación

Se divide la ubicación por 
* Colonia
* Municipio
* Estado

In [130]:
# Split 'ubicacion' into a maximum of 3 parts and handle missing values
df[['Colonia', 'Municipio', 'Estado']] = df['ubicacion'].str.split(',', expand=True, n=2)

# Clean spaces around the values
df['Colonia'] = df['Colonia'].str.strip()
df['Municipio'] = df['Municipio'].str.strip()
df['Estado'] = df['Estado'].str.strip()

# Fill NaN or missing values with specific defaults
df['Colonia'] = df['Colonia'].fillna('Sin_colonia')
df['Municipio'] = df['Municipio'].fillna('Sin_municipio')
df['Estado'] = df['Estado'].fillna('Sin_estado')

# Show the resulting DataFrame
df[['ubicacion', 'Colonia', 'Municipio', 'Estado']]


,ubicacion,Colonia,Municipio,Estado
0,"Jurica Misiones, Juriquilla, QuerÃ©taro Arteaga",Jurica Misiones,Juriquilla,QuerÃ©taro Arteaga
1,"Cumbres del Lago, Juriquilla, QuerÃ©taro Arteaga",Cumbres del Lago,Juriquilla,QuerÃ©taro Arteaga
2,"Villas de Santiago, QuerÃ©taro, QuerÃ©taro Arteaga",Villas de Santiago,QuerÃ©taro,QuerÃ©taro Arteaga
3,"Cumbres del Lago, Juriquilla, QuerÃ©taro Arteaga",Cumbres del Lago,Juriquilla,QuerÃ©taro Arteaga
4,"Cumbres del Lago, Juriquilla, QuerÃ©taro Arteaga",Cumbres del Lago,Juriquilla,QuerÃ©taro Arteaga
...,...,...,...,...
10993,"Santa FÃ©, Juriquilla, QuerÃ©taro Arteaga",Santa FÃ©,Juriquilla,QuerÃ©taro Arteaga
10994,"Cumbres del Lago, Juriquilla, QuerÃ©taro Arteaga",Cumbres del Lago,Juriquilla,QuerÃ©taro Arteaga
10995,"El Refugio, QuerÃ©taro, QuerÃ©taro Arteaga",El Refugio,QuerÃ©taro,QuerÃ©taro Arteaga
10996,"La Cima, Santiago de QuerÃ©taro, QuerÃ©taro Arteaga",La Cima,Santiago de QuerÃ©taro,QuerÃ©taro Arteaga


In [131]:
df['Colonia'].unique()

array(['Jurica Misiones', 'Cumbres del Lago', 'Villas de Santiago',
       'Sonterra', 'Las Fuentes', 'La Cima',
       'SatÃ©lite SecciÃ³n Condominios', 'El Rocio', 'La Joya',
       'Colinas del Cimatario', 'Los Fundadores',
       'Santiago de QuerÃ©taro Centro', 'Santa FÃ©', 'Las Plazas',
       'Unidad Roma', 'Los Ciruelos', 'El Refugio', 'Del MesÃ³n',
       'Santa Rosa de Jauregui', 'Ciudad de QuerÃ©taro', 'Balcones',
       'Lomas de Casa Blanca', 'Galindas', 'Privada Juriquilla',
       'Santa MarÃ\xada', 'Jurica Pinar', 'El Sol', 'Arboledas',
       'Campanario Miradores', 'Montenegro', 'Bosques de QuerÃ©taro',
       'El Mirador', 'Milenio III', 'La Tinaja', 'Universo 200',
       'Las Palmas', 'San Mateo', 'Lomas del MarquÃ©s 1 y 2 Etapa',
       'San Pedrito PeÃ±uelas II', 'Lomas de San Pedrito',
       'Jardines de La Hacienda', 'El Jacal', 'San JosÃ© El Alto',
       'Menchaca', 'Prados de La Capilla', 'San Pablo', 'Cipreses',
       'Claustros de la Loma', 'Vista Alegre

## c) Estandarizar valores Colonia

In [132]:
# Convierte todos los valores a strings, maneja NaN o None como cadenas vacías
df['Colonia'] = df['Colonia'].astype(str)
# Convertir a minúsculas
df['Colonia'] = df['Colonia'].str.lower()

import unicodedata# Función para eliminar acentos
def eliminar_acentos(texto):
    # Normaliza y elimina los acentos
    texto_normalizado = unicodedata.normalize('NFKD', texto)
    return ''.join(char for char in texto_normalizado if unicodedata.category(char) != 'Mn')

# Aplicar la función de eliminación de acentos
df['Colonia'] = df['Colonia'].apply(eliminar_acentos)

# Reemplazar Ñ
df['Colonia'] = df['Colonia'].str.replace('ñ', 'n')
df['Colonia'] = df['Colonia'].str.replace(r'[^a-zA-Z0-9\s]', '', regex=True)

## d) Mapeo de CP por Excel

In [133]:
# Cargar todas las hojas del archivo Excel en un diccionario de DataFrames
ruta_excel = 'D:\yoe11\Documents\TESEO\Modelo Factible\Scraplining\CPdescarga.xlsx'  
hojas = pd.read_excel(ruta_excel, sheet_name=None)

# Obtener combinaciones únicas de Estado, Municipio y Colonia
combinaciones_unicas = df[['Estado', 'Municipio', 'Colonia']].drop_duplicates()

# Crear un diccionario para almacenar los códigos postales encontrados
codigos_postales = {}

# Buscar el código postal para cada combinación única
for _, row in combinaciones_unicas.iterrows():
    estado = row['Estado'].strip()
    municipio = row['Municipio'].strip()
    colonia = row['Colonia'].strip()

    # Verificar si el estado existe en las hojas del Excel
    if estado in hojas:
        df_estado = hojas[estado]

        # Filtrar por municipio y colonia
        df_filtrado = df_estado[
            (df_estado['D_mnpio'].str.strip().str.lower() == municipio.lower()) &
            (df_estado['d_asenta'].str.strip().str.lower() == colonia.lower())  # Suponiendo que la columna de colonia se llama 'd_asenta'
        ]

        # Si hay coincidencias, almacenar el primer código postal en el diccionario
        if not df_filtrado.empty:
            codigos_postales[(estado, municipio, colonia)] = df_filtrado['d_codigo'].iloc[0]
        else:
            codigos_postales[(estado, municipio, colonia)] = None
    else:
        codigos_postales[(estado, municipio, colonia)] = None

# Crear una columna 'Codigo_Postal' en el DataFrame original usando el diccionario de códigos postales
df['Codigo_Postal'] = df.apply(lambda x: codigos_postales.get((x['Estado'].strip(), x['Municipio'].strip(), x['Colonia'].strip())), axis=1)


# Obtener las combinaciones únicas de las columnas 'Colonia', 'Municipio', 'Estado', 'Codigo_Postal'
combinaciones_unicas = df[['Colonia', 'Municipio', 'Estado', 'Codigo_Postal']].drop_duplicates()
for idx, row in combinaciones_unicas.iterrows():
    print(f"Combinación {idx + 1}:")
    print(f"Colonia: {row['Colonia']}, Municipio: {row['Municipio']}, Estado: {row['Estado']}, Codigo_Postal: {row['Codigo_Postal']}")
    print("---")


ValueError: Value must be either numerical or a string containing a wildcard

## e) Fraccionamientos

In [20]:
# Traer de la base original de los cp, identificar la colonia si es fraccionamiento, 
# si lo verifica que se le asigne un valor para identificar cuál es cual.

# Cargar todas las hojas del archivo Excel en un diccionario de DataFrames
ruta_excel = 'D:\yoe11\Documents\TESEO\Modelo Factible\Scraplining\CPdescarga.xlsx'  
hojas = pd.read_excel(ruta_excel, sheet_name=None)

# Obtener combinaciones únicas de Estado, Municipio y Colonia
combinaciones_unicas = df[['Estado', 'Municipio', 'Colonia']].drop_duplicates()

# Crear un diccionario para almacenar los códigos postales y tipos de asentamiento encontrados
codigos_postales = {}
tipos_asentamiento = {}

# Buscar el código postal y tipo de asentamiento para cada combinación única
for _, row in combinaciones_unicas.iterrows():
    estado = row['Estado'].strip()
    municipio = row['Municipio'].strip()
    colonia = row['Colonia'].strip()

    # Verificar si el estado existe en las hojas del Excel
    if estado in hojas:
        df_estado = hojas[estado]

        # Filtrar por municipio y colonia
        df_filtrado = df_estado[
            (df_estado['D_mnpio'].str.strip().str.lower() == municipio.lower()) &
            (df_estado['d_asenta'].str.strip().str.lower() == colonia.lower())
        ]

        # Si hay coincidencias, almacenar el primer código postal y tipo de asentamiento en los diccionarios
        if not df_filtrado.empty:
            codigos_postales[(estado, municipio, colonia)] = df_filtrado['d_codigo'].iloc[0]
            tipos_asentamiento[(estado, municipio, colonia)] = df_filtrado['d_tipo_asenta'].iloc[0]
        else:
            codigos_postales[(estado, municipio, colonia)] = None
            tipos_asentamiento[(estado, municipio, colonia)] = None
    else:
        codigos_postales[(estado, municipio, colonia)] = None
        tipos_asentamiento[(estado, municipio, colonia)] = None

# Crear columnas 'Codigo_Postal' y 'Tipo_Asentamiento' en el DataFrame original usando los diccionarios
df['Codigo_Postal'] = df.apply(lambda x: codigos_postales.get((x['Estado'].strip(), x['Municipio'].strip(), x['Colonia'].strip())), axis=1)
df['Tipo_Asentamiento'] = df.apply(lambda x: tipos_asentamiento.get((x['Estado'].strip(), x['Municipio'].strip(), x['Colonia'].strip())), axis=1)

# Obtener las combinaciones únicas de las columnas 'Colonia', 'Municipio', 'Estado', 'Codigo_Postal', 'Tipo_Asentamiento'
combinaciones_unicas = df[['Colonia', 'Municipio', 'Estado', 'Codigo_Postal', 'Tipo_Asentamiento']].drop_duplicates()
for idx, row in combinaciones_unicas.iterrows():
    print(f"Combinación {idx + 1}:")
    print(f"Colonia: {row['Colonia']}, Municipio: {row['Municipio']}, Estado: {row['Estado']}, Codigo_Postal: {row['Codigo_Postal']}, Tipo_Asentamiento: {row['Tipo_Asentamiento']}")
    print("---")

ValueError: Value must be either numerical or a string containing a wildcard

In [ ]:
## E

___
# Eliminar variables extra

In [ ]:
df = df.drop(['precio_desc','totales_desc','construidos_desc','estacionamiento_desc','recamaras_desc','baños_desc','baños_completos','medios_baños'], axis=1) # 'denominacion_desc'

# EXPORTAR

In [162]:
# Lista de nombres de columnas en el orden deseado
#column_order = ['precio', 'denominacion', 'propiedad', 'metros_total', 'metros_construido', 'tiempo_de_publicacion', 'meses_2024-09-06', 'tipo', 'estacionamientos', 'recamaras', 'banos', 'medio_banos', 'seguridad_privada', 'Estado', 'Municipio', 'Colonia', 'Codigo_Postal', 'url', 'descripcion']

# Reordenar el DataFrame según el orden de columnas deseado
#df = df[column_order]

# Exportar el DataFrame a un archivo CSV con codificación UTF-8
df.to_csv('may_queretaro_Inmuebles_2024.csv', encoding='utf-8', index=False)